# Data Cleaning Pipeline

This notebook executes the data importing, merging and cleaning process using the `data_cleaning` module.

External imports

In [1]:
import os 
import pandas as pd
import pathlib
import numpy as np
from tqdm import tqdm

Internal import

In [2]:
from data_cleaning.process_charts import process_all_charts
from data_cleaning.merge import merge_data
from data_cleaning.clean_songs import (
    list_weekly_chart_files,
    extract_dates_from_filenames,
    summarize_weekly_date_gaps,
    create_song_dict,
    update_song_rows_with_dict,
    fill_with_proxy_dict_compat,
    fill_missing_from_dfs,
    prepare_df_for_parquet,
)


Data import

In [3]:
DATA_DIR = "data"

### Creating files' paths
# Bronze files
weekly_charts_path = os.path.join(DATA_DIR, "bronze", "data")
tracks_path = os.path.join(DATA_DIR, "bronze", "tracks.csv")

# Silver files
songs_path = os.path.join(DATA_DIR, "silver", "combined_songs.parquet")
output_path = os.path.join(DATA_DIR, "silver", "songs_with_features.parquet")

### Merging our csvs' into parquet files in Silver for us to work with
print("Starting data processing...")
process_all_charts(weekly_charts_path, songs_path)

print("Data processing complete.")

print("Starting data merging...")
merge_data(tracks_path, songs_path, output_path)
print("Data merging complete.")


if os.path.exists(output_path):
    # read parquet
    songs = pd.read_parquet(output_path)
    print("Songs loaded successfully. ({:_} rows)".format(songs.shape[0]))
else:
    raise FileNotFoundError("Error: Output path does not exist.")

Starting data processing...
✓ File saved as: data/silver/combined_songs.parquet
Data processing complete.
Starting data merging...
✓ Merged data saved to: data/silver/songs_with_features.parquet
Data merging complete.
Songs loaded successfully. (41_995 rows)


## Preliminary quality check
Firt, we want to check that our dataset spans all the desired weeks.

In [4]:
# Verification done through helper functions from data_cleaning.clean_songs
files = list_weekly_chart_files(weekly_charts_path)
dates = extract_dates_from_filenames(files)
summarize_weekly_date_gaps(dates)

First date: 2016-12-29
Last date: 2020-12-31
Total files: 210
Expected weeks: 210

Missing weeks:

Unexpected extra dates:


Then we want to check our data quality by checking wether we are missing any values.

In [5]:
# Check if there are empty values in the dataset
print(songs.isna().sum())


track_id                0
artist_names            0
track_name              0
source                  0
streams                 0
week_date               0
name                10631
popularity          10631
duration_ms         10631
explicit            10631
artists             10631
id_artists          10631
release_date        10631
danceability        10631
energy              10631
key                 10631
loudness            10631
mode                10631
speechiness         10631
acousticness        10631
instrumentalness    10631
liveness            10631
valence             10631
tempo               10631
time_signature      10631
dtype: int64


Here we notice that we are missing values, but it is always the same number, so we can suppose that this issue comes from the merging of our scrapped file with the kaggle file.


## In depth data cleaning
### Deleting useless data

First we want to see our dataset's features, such as its columns and shape.

In [6]:
songs.columns, songs.shape

(Index(['track_id', 'artist_names', 'track_name', 'source', 'streams',
        'week_date', 'name', 'popularity', 'duration_ms', 'explicit', 'artists',
        'id_artists', 'release_date', 'danceability', 'energy', 'key',
        'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
        'liveness', 'valence', 'tempo', 'time_signature'],
       dtype='object'),
 (41995, 25))

Then, we notice that the columns "name" and "track_name" are supposed to contain the same values, but were not identified as foreign keys within the join. Therefore, we check if they both indeed contain the same information and therefore if we can remove one of them to not have duplicates.

In [7]:
def drop_column_if_exists(df, column):
    """
    Drops the specified column from the DataFrame if it exists.
    Returns the modified DataFrame.
    """
    if column in df.columns:
        print(f"Rows where {column} is not included in track_name: {df[df[column].notna() & df['track_name'].isna()].shape[0]}/{df.shape[0]}")
        df = df.drop(columns=[column])
        print(f"Column '{column}' dropped successfully.")
    return df

songs = drop_column_if_exists(songs, "name")

Rows where name is not included in track_name: 0/41995
Column 'name' dropped successfully.


Now, we can recheck our columns and name to check that we haven't damaged the dataset.

In [8]:
songs.columns, songs.shape

(Index(['track_id', 'artist_names', 'track_name', 'source', 'streams',
        'week_date', 'popularity', 'duration_ms', 'explicit', 'artists',
        'id_artists', 'release_date', 'danceability', 'energy', 'key',
        'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
        'liveness', 'valence', 'tempo', 'time_signature'],
       dtype='object'),
 (41995, 24))

## Create unique IDs

We also notice, when looking in our dataset, that we sometimes have the same songs in the charts, but under different track_ID. That is because sometimes a singer would publish the same music multiple times (firstly as an EP or a mixtape, then in an album, then in a best of etc...). But in the case of our study, it is important that these songs have the same ID as they are the same music. 

Therefore we identify the songs and unify their track_IDs.

First we create a dictionnary of all the songs with similar artists and title and associate them with a unique ID.

In [9]:
# Create a dictionary of canonical song IDs using helper from data_cleaning.clean_songs
song_dict = create_song_dict(songs)
print(song_dict[("The Weeknd", "Blinding Lights")])

Processing rows: 100%|██████████| 41995/41995 [00:00<00:00, 60911.92it/s]

['0VjIjW4GlUZAMYd2vXMi3b', 'Republic Records', Timestamp('2020-03-20 00:00:00')]


Then we update the IDs within the dataset, so that every song has its own ID.

In [10]:
# Apply the update function from data_cleaning.clean_songs
songs = update_song_rows_with_dict(songs, song_dict)

Updating songs: 100%|██████████| 41995/41995 [00:05<00:00, 8201.54it/s]

Number of songs updated: 6_458/41_995


Using these new unique ID, we can fill the information about song that had different IDs but were the similar music, that had been ignored by the merge.

In [11]:
# Fill missing values in columns of interest using helpers from data_cleaning.clean_songs
columns_to_fill = [
    'artist_names', 'track_name', 'source', 'duration_ms', 'explicit', 
    'popularity', 'artists', 'id_artists', 'release_date', 'danceability', 'energy', 
    'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
    'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'
]

# If we already have values for a track somewhere and in another orw it is NaN, we fill it with what we have
songs = fill_with_proxy_dict_compat(songs, columns_to_fill)


Number of rows filled: 11_069


In [12]:
print("There are still {:_} missing values in the DataFrame".format(songs.isna().sum().sum()))

There are still 125_826 missing values in the DataFrame


## Complete our missing data through another dataset

We notice that there are still songs which are missing some data. Therefore we decide to enrich those values through other diverse datasets, found on kaggle, that also extract the official spotify audio features.

In [13]:
### Import the new dataset

#Create the paths
df_enrichment2_path = os.path.join(DATA_DIR, "bronze", "spotify_top_songs_audio_features.csv")
df_enrichment2 = pd.read_csv(df_enrichment2_path)


The column ID will serve as a foreign key, so we rename the column ID by Track_ID, and fill the values so that the same songs have the same ID.

In [14]:
#Correspondance between the IDs
if "id" in df_enrichment2.columns:
    # Replace id with track_id
    df_enrichment2.rename(columns={"id": "track_id"}, inplace=True)
df_enrichment2.shape, df_enrichment2.columns 

((6513, 19),
 Index(['track_id', 'artist_names', 'track_name', 'source', 'key', 'mode',
        'time_signature', 'danceability', 'energy', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'loudness',
        'tempo', 'duration_ms', 'weeks_on_chart', 'streams'],
       dtype='object'))

In [15]:
### Import and merge all the dataset within the enrichment folder. (no merging issue as they all follow the same format)

# List all dataframes in a kaggle_enrichment3_dir, then add them together to it is one big dataframe
kaggle_enrichment3_dir = os.path.join(DATA_DIR, "bronze", "kaggle_enrichment3")
import glob

# List all CSV files in the kaggle_enrichment3_dir
csv_files = glob.glob(os.path.join(kaggle_enrichment3_dir, "*.csv"))

# Read each CSV file into a DataFrame and collect them in a list
df_list = [pd.read_csv(f) for f in csv_files]

# Concatenate all DataFrames into a single big DataFrame
df_enrichment3 = pd.concat(df_list, ignore_index=True)

# Show shape and columns to confirm final structure
df_enrichment3.shape, df_enrichment3.columns


((247035, 17),
 Index(['artist_name', 'track_id', 'track_name', 'acousticness', 'danceability',
        'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
        'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence',
        'popularity'],
       dtype='object'))

Then, we check that the columns we need to fill can indeed be filled by these new dataset. 

To do so we compare the list of columns to fill with the columns in our enrichment dataset.

In [16]:
set(df_enrichment2.columns).difference(set(columns_to_fill)), set(df_enrichment3.columns).difference(set(columns_to_fill))

({'streams', 'track_id', 'weeks_on_chart'}, {'artist_name', 'track_id'})

Here we observe that 'streams', 'weeks_on_chart' are absent from the second enrichment dataset, and that 'artist_name' is absent from the third (track_id being considered absent as the column is named ID in the dataframe, but as it will be used as a foreign key, it poses no issue for the completion).

Since there is no column missing from both dataset, we can use these to fill our missing values.

In terms of data quality, we know they are also official music features from spotify, so they have the same format as the data we already have within our original dataset. (And we checked in the datasets' documentations just in case)

So now we can now fill our missing values !

In [17]:
# Enrich missing values from external enrichment DataFrames using helper from data_cleaning.clean_songs
songs_gold = fill_missing_from_dfs(songs, columns_to_fill, "track_id", df_enrichment2, df_enrichment3)

Total missing values *before* processing DF: 125_826
Available columns: ['artist_names', 'track_name', 'source', 'duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
Size of lookup dictionary : 6513


Enriching songs: 100%|██████████| 41995/41995 [00:12<00:00, 3359.79it/s]


Available columns: ['track_name', 'duration_ms', 'popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
Size of lookup dictionary : 130989


Enriching songs: 100%|██████████| 41995/41995 [00:03<00:00, 11736.34it/s]


Total missing values *after* processing DF: 33_889


Now we can check that our filling has gone smoothly and check our NA values again.

In [18]:
print(songs_gold.isna().sum())

track_id               0
artist_names           0
track_name             0
source                 0
streams                0
week_date              0
popularity          5587
duration_ms            0
explicit            6966
artists             6966
id_artists          6966
release_date        7404
danceability           0
energy                 0
key                    0
loudness               0
mode                   0
speechiness            0
acousticness           0
instrumentalness       0
liveness               0
valence                0
tempo                  0
time_signature         0
dtype: int64


A lot of our NA values come from the column release date. Since it won't be really useful in our analysis, we can simply remove it. 

Similarly, the measure popularity seems in our case useless for the analysis since it does not relate to audio features. 

Therefore we can drop our columns.

In [19]:
# We get rid of popularity and release_date as they will not be used for further analysis

if "popularity" in songs_gold.columns:
    songs_gold.drop(columns=["popularity"], inplace=True)
    
if "release_date" in songs_gold.columns:
    songs_gold.drop(columns=["release_date"], inplace=True)


Another check of our dataset.

In [20]:
# Display the number of NaN values for each column in songs_gold
print(songs_gold.isna().sum())

track_id               0
artist_names           0
track_name             0
source                 0
streams                0
week_date              0
duration_ms            0
explicit            6966
artists             6966
id_artists          6966
danceability           0
energy                 0
key                    0
loudness               0
mode                   0
speechiness            0
acousticness           0
instrumentalness       0
liveness               0
valence                0
tempo                  0
time_signature         0
dtype: int64


While we could do our analysis without the artists' data (although it would be less practical to identify tracks), the feature explicit is interesting to our study. 
Therefore, we want to check if the rows that are missing the explicit data are the same as the others with NaN. 

In [21]:
songs_gold_witouht_nan = songs_gold.dropna()
print(f"Number of total rows with NaN values {songs_gold.shape[0] - songs_gold_witouht_nan.shape[0]}")
print("Complete Rows / Total Rows: {}/{}".format(songs_gold_witouht_nan.shape[0], songs_gold.shape[0]))


Number of total rows with NaN values 6966
Complete Rows / Total Rows: 35029/41995


Since the number of total rows with missing values is the same than the number of rows with missing values in the "explicit" column, it means that all missing values are within the same rows.

## Save gold dataset

As we want to keep "explicit", we decide to keep the dataset without the rows where it is empty.

Let's check that this dataset is not missing any value:

In [22]:
print(songs_gold_witouht_nan.isna().sum())

track_id            0
artist_names        0
track_name          0
source              0
streams             0
week_date           0
duration_ms         0
explicit            0
artists             0
id_artists          0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
dtype: int64


No NaN ! 

We can therefore save our clean dataset to reuse it in our visualisation notebook.

In [23]:
# Prepare dataframe for parquet (handles type conversions)
songs_gold_witouht_nan_parquet = prepare_df_for_parquet(songs_gold_witouht_nan)

songs_gold_witouht_nan_parquet.to_parquet(os.path.join(DATA_DIR, "gold", "songs_with_features.parquet"), index=False)


You can find here commented python lines that we wanted to use to fill in the explicit column. As it employs a private API, we decided that dropping the rows was better for this exercise. 

However, we wanted to keep our solution for the record, in case this analysis is used again in the future.

In [24]:
# # Uncomment to use Gemini to enrich explicit values
# from data_cleaning.artist_mapping import (
#     get_unique_ids_from_column,
#     get_all_combinations,
#     get_artist_to_id,
#     update_id_artists_with_mapping,
# )
# from data_cleaning.explicit_enrichment import (
#     gemini_check_if_explicit,
#     enrich_explicit_via_gemini,
# )

In [25]:
# Functions moved to data_cleaning.artist_mapping module
# (get_all_combinations, get_artist_to_id, update_id_artists_with_mapping)